# Base Preprocessing

## Import Libraries and Loading Data

In [149]:
# Explore dataset
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Unicode normalization
import unicodedata

# Handling encoding and imputation
import category_encoders as ce
from feature_engine.encoding import CountFrequencyEncoder # Frecuency encoding
from sklearn.preprocessing import OrdinalEncoder, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.base import BaseEstimator, TransformerMixin # Custom preprocessing steps

# Creating a pipeline for more reliable preprocessing process
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

In [150]:
# Load data
DATA_DIR = "datasets"

df_full = pd.read_csv(f"{DATA_DIR}/train.csv")
df_full.head()

,ID,PERIODO,ESTU_PRGM_ACADEMICO,ESTU_PRGM_DEPARTAMENTO,ESTU_VALORMATRICULAUNIVERSIDAD,ESTU_HORASSEMANATRABAJA,FAMI_ESTRATOVIVIENDA,FAMI_TIENEINTERNET,FAMI_EDUCACIONPADRE,FAMI_TIENELAVADORA,FAMI_TIENEAUTOMOVIL,ESTU_PRIVADO_LIBERTAD,ESTU_PAGOMATRICULAPROPIO,FAMI_TIENECOMPUTADOR,FAMI_TIENEINTERNET.1,FAMI_EDUCACIONMADRE,RENDIMIENTO_GLOBAL
0,904256,20212,ENFERMERIA,BOGOTÁ,Entre 5.5 millones y menos de 7 millones,Menos de 10 horas,Estrato 3,Si,Técnica o tecnológica incompleta,Si,Si,N,No,Si,Si,Postgrado,medio-alto
1,645256,20212,DERECHO,ATLANTICO,Entre 2.5 millones y menos de 4 millones,0,Estrato 3,No,Técnica o tecnológica completa,Si,No,N,No,Si,No,Técnica o tecnológica incompleta,bajo
2,308367,20203,MERCADEO Y PUBLICIDAD,BOGOTÁ,Entre 2.5 millones y menos de 4 millones,Más de 30 horas,Estrato 3,Si,Secundaria (Bachillerato) completa,Si,No,N,No,No,Si,Secundaria (Bachillerato) completa,bajo
3,470353,20195,ADMINISTRACION DE EMPRESAS,SANTANDER,Entre 4 millones y menos de 5.5 millones,0,Estrato 4,Si,No sabe,Si,No,N,No,Si,Si,Secundaria (Bachillerato) completa,alto
4,989032,20212,PSICOLOGIA,ANTIOQUIA,Entre 2.5 millones y menos de 4 millones,Entre 21 y 30 horas,Estrato 3,Si,Primaria completa,Si,Si,N,No,Si,Si,Primaria completa,medio-bajo


In [151]:
df_full.describe(include="object").T

,count,unique,top,freq
ESTU_PRGM_ACADEMICO,692500,948,DERECHO,53244
ESTU_PRGM_DEPARTAMENTO,692500,31,BOGOTÁ,282159
ESTU_VALORMATRICULAUNIVERSIDAD,686213,8,Entre 1 millón y menos de 2.5 millones,204048
ESTU_HORASSEMANATRABAJA,661643,5,Más de 30 horas,249352
FAMI_ESTRATOVIVIENDA,660363,7,Estrato 2,232671
FAMI_TIENEINTERNET,665871,2,Si,592514
FAMI_EDUCACIONPADRE,669322,12,Secundaria (Bachillerato) completa,128289
FAMI_TIENELAVADORA,652727,2,Si,563390
FAMI_TIENEAUTOMOVIL,648877,2,No,412606
ESTU_PRIVADO_LIBERTAD,692500,2,N,692466


In [152]:
# Drop columns that we do not need for preprocessing
df_train = df_full.drop(columns=["ID", "RENDIMIENTO_GLOBAL", "FAMI_TIENEINTERNET.1"])

## Normalization

In [153]:
def normalize_text(text):
    text = unicodedata.normalize("NFKD", text)
    text = text.encode("ASCII", "ignore")
    return text.decode("UTF-8")

def normalize_data(df):
    # Normalize "periodo", change dtype to str and extract the year
    df["PERIODO"] = df["PERIODO"].astype(str).apply(lambda text: text[:4])
    # Normalize "programa", remove weird characters in the text
    df["ESTU_PRGM_ACADEMICO"] = df_train["ESTU_PRGM_ACADEMICO"].apply(normalize_text)

    return df

In [154]:
df_train = normalize_data(df_train)
df_train.head()

,PERIODO,ESTU_PRGM_ACADEMICO,ESTU_PRGM_DEPARTAMENTO,ESTU_VALORMATRICULAUNIVERSIDAD,ESTU_HORASSEMANATRABAJA,FAMI_ESTRATOVIVIENDA,FAMI_TIENEINTERNET,FAMI_EDUCACIONPADRE,FAMI_TIENELAVADORA,FAMI_TIENEAUTOMOVIL,ESTU_PRIVADO_LIBERTAD,ESTU_PAGOMATRICULAPROPIO,FAMI_TIENECOMPUTADOR,FAMI_EDUCACIONMADRE
0,2021,ENFERMERIA,BOGOTÁ,Entre 5.5 millones y menos de 7 millones,Menos de 10 horas,Estrato 3,Si,Técnica o tecnológica incompleta,Si,Si,N,No,Si,Postgrado
1,2021,DERECHO,ATLANTICO,Entre 2.5 millones y menos de 4 millones,0,Estrato 3,No,Técnica o tecnológica completa,Si,No,N,No,Si,Técnica o tecnológica incompleta
2,2020,MERCADEO Y PUBLICIDAD,BOGOTÁ,Entre 2.5 millones y menos de 4 millones,Más de 30 horas,Estrato 3,Si,Secundaria (Bachillerato) completa,Si,No,N,No,No,Secundaria (Bachillerato) completa
3,2019,ADMINISTRACION DE EMPRESAS,SANTANDER,Entre 4 millones y menos de 5.5 millones,0,Estrato 4,Si,No sabe,Si,No,N,No,Si,Secundaria (Bachillerato) completa
4,2021,PSICOLOGIA,ANTIOQUIA,Entre 2.5 millones y menos de 4 millones,Entre 21 y 30 horas,Estrato 3,Si,Primaria completa,Si,Si,N,No,Si,Primaria completa


In [155]:
df_train.nunique()

PERIODO                             4
ESTU_PRGM_ACADEMICO               787
ESTU_PRGM_DEPARTAMENTO             31
ESTU_VALORMATRICULAUNIVERSIDAD      8
ESTU_HORASSEMANATRABAJA             5
FAMI_ESTRATOVIVIENDA                7
FAMI_TIENEINTERNET                  2
FAMI_EDUCACIONPADRE                12
FAMI_TIENELAVADORA                  2
FAMI_TIENEAUTOMOVIL                 2
ESTU_PRIVADO_LIBERTAD               2
ESTU_PAGOMATRICULAPROPIO            2
FAMI_TIENECOMPUTADOR                2
FAMI_EDUCACIONMADRE                12
dtype: int64

## Missing Values and Duplicated Columns

In [156]:
df_train.isnull().sum().sort_values(ascending=False)

FAMI_TIENEAUTOMOVIL               43623
FAMI_TIENELAVADORA                39773
FAMI_TIENECOMPUTADOR              38103
FAMI_ESTRATOVIVIENDA              32137
ESTU_HORASSEMANATRABAJA           30857
FAMI_TIENEINTERNET                26629
FAMI_EDUCACIONMADRE               23664
FAMI_EDUCACIONPADRE               23178
ESTU_PAGOMATRICULAPROPIO           6498
ESTU_VALORMATRICULAUNIVERSIDAD     6287
PERIODO                               0
ESTU_PRGM_ACADEMICO                   0
ESTU_PRGM_DEPARTAMENTO                0
ESTU_PRIVADO_LIBERTAD                 0
dtype: int64

In [157]:
# Calculate the percentage of missing values in each feature
missing_values = df_train.isnull().sum()
total_cells = df_train.shape[0]
missing_percentage = (missing_values / total_cells) * 100
missing_percentage.sort_values(ascending=False)

FAMI_TIENEAUTOMOVIL               6.299350
FAMI_TIENELAVADORA                5.743394
FAMI_TIENECOMPUTADOR              5.502238
FAMI_ESTRATOVIVIENDA              4.640722
ESTU_HORASSEMANATRABAJA           4.455884
FAMI_TIENEINTERNET                3.845343
FAMI_EDUCACIONMADRE               3.417184
FAMI_EDUCACIONPADRE               3.347004
ESTU_PAGOMATRICULAPROPIO          0.938339
ESTU_VALORMATRICULAUNIVERSIDAD    0.907870
PERIODO                           0.000000
ESTU_PRGM_ACADEMICO               0.000000
ESTU_PRGM_DEPARTAMENTO            0.000000
ESTU_PRIVADO_LIBERTAD             0.000000
dtype: float64

In [158]:
# Count the duplicated values
# df_train[["FAMI_TIENEINTERNET", "FAMI_TIENEINTERNET.1"]].duplicated().value_counts()

Since we a duplicated feature, we are going to drop it to avoid noise in futures models.

## Imputing and Encoding

Since missing percentage in each feature (column) in our dataset is low, imputation is a better option to process missing values.

For imputation, given that our variables are mostly categorical we have two strategies
- Replacing all missing values with `Unknown` (We are going to use this one for the base)
- Replacing all missing values with the `most frequent` value

For encoding, we are doing
- Frequency encoding for features with "High-Cardinality"
- Onehot encoding for binominal and nominal features
- Ordinal encoding for `FAMI_ESTRATOFAMILIA` since it describes a status class


In [159]:
# Select variables
ordinal_cols = ["FAMI_ESTRATOVIVIENDA"]
high_card_cols = [col for col in df_train.columns if df_train[col].nunique() > 15]
cat_cols = [col for col in df_train.columns if col not in (ordinal_cols + high_cardinality_cols)]

In [160]:
# Custom categories for Ordinal Variables
custom_categories = [
    ["Sin Estrato", "Estrato 1", "Estrato 2", "Estrato 3", "Estrato 4", "Estrato 5", "Estrato 6"]
]

# Create all processes to preprocess the data
ordinal_transformer = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="constant", fill_value="Sin Estrato")),
    ("encoder", OrdinalEncoder(categories=custom_categories))
])

cat_transformer = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="constant", fill_value="Unknown")),
    ("encoder", OneHotEncoder())
])

high_card_transformer = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="constant", fill_value="Unknown")),
    ("encoder", CountFrequencyEncoder())
])

## Preprocessing

In [161]:
preprocessor = ColumnTransformer(
    transformers=[
        ("ord", ordinal_transformer, ordinal_cols),
        ("cat", cat_transformer, cat_cols)
    ]
)

In [162]:
# TODO: Output the data preprocessed

In [163]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split

model = DecisionTreeClassifier(random_state=42)

pipe = Pipeline(steps=[
    ("preprocessing", preprocessor),
    ("model", model)
])

In [164]:
X = df_full.drop(["ID", "RENDIMIENTO_GLOBAL", "FAMI_TIENEINTERNET.1"], axis=1)
y = df_full.RENDIMIENTO_GLOBAL

In [165]:
X = normalize_data(X)
X.columns

Index(['PERIODO', 'ESTU_PRGM_ACADEMICO', 'ESTU_PRGM_DEPARTAMENTO',
       'ESTU_VALORMATRICULAUNIVERSIDAD', 'ESTU_HORASSEMANATRABAJA',
       'FAMI_ESTRATOVIVIENDA', 'FAMI_TIENEINTERNET', 'FAMI_EDUCACIONPADRE',
       'FAMI_TIENELAVADORA', 'FAMI_TIENEAUTOMOVIL', 'ESTU_PRIVADO_LIBERTAD',
       'ESTU_PAGOMATRICULAPROPIO', 'FAMI_TIENECOMPUTADOR',
       'FAMI_EDUCACIONMADRE'],
      dtype='object')

In [166]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

pipe.fit(X_train, y_train)

Pipeline(steps=[('preprocessing',
                 ColumnTransformer(transformers=[('ord',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(fill_value='Sin '
                                                                                            'Estrato',
                                                                                 strategy='constant')),
                                                                  ('encoder',
                                                                   OrdinalEncoder(categories=[['Sin '
                                                                                               'Estrato',
                                                                                               'Estrato '
                                                                                               '1',
                                                                                               'Estrato '
                                                                                               '2',
                                                                                               'Estrato '
                                                                                               '3',
                                                                                               'Estrato '
                                                                                               '4',
                                                                                               'Estrato '
                                                                                               '5',
                                                                                               'Estrato '
                                                                                               '6']]))]),
                                                  ['FAMI_ESTRATOVIVIENDA']),
                                                 ('cat',
                                                  Pipeline(steps=[('...
                                                                                 strategy='constant')),
                                                                  ('encoder',
                                                                   OneHotEncoder())]),
                                                  ['PERIODO',
                                                   'ESTU_VALORMATRICULAUNIVERSIDAD',
                                                   'ESTU_HORASSEMANATRABAJA',
                                                   'FAMI_TIENEINTERNET',
                                                   'FAMI_EDUCACIONPADRE',
                                                   'FAMI_TIENELAVADORA',
                                                   'FAMI_TIENEAUTOMOVIL',
                                                   'ESTU_PRIVADO_LIBERTAD',
                                                   'ESTU_PAGOMATRICULAPROPIO',
                                                   'FAMI_TIENECOMPUTADOR',
                                                   'FAMI_EDUCACIONMADRE'])])),
                ('model', DecisionTreeClassifier(random_state=42))])

In [167]:
X_test

,PERIODO,ESTU_PRGM_ACADEMICO,ESTU_PRGM_DEPARTAMENTO,ESTU_VALORMATRICULAUNIVERSIDAD,ESTU_HORASSEMANATRABAJA,FAMI_ESTRATOVIVIENDA,FAMI_TIENEINTERNET,FAMI_EDUCACIONPADRE,FAMI_TIENELAVADORA,FAMI_TIENEAUTOMOVIL,ESTU_PRIVADO_LIBERTAD,ESTU_PAGOMATRICULAPROPIO,FAMI_TIENECOMPUTADOR,FAMI_EDUCACIONMADRE
137278,2019,ADMINISTRACION POLICIAL,BOGOTÁ,Entre 2.5 millones y menos de 4 millones,Entre 21 y 30 horas,Estrato 2,Si,Secundaria (Bachillerato) completa,Si,Si,N,No,Si,Educación profesional completa
534550,2019,MEDICINA,RISARALDA,Entre 1 millón y menos de 2.5 millones,0,Estrato 4,Si,Educación profesional completa,Si,Si,N,Si,Si,Educación profesional completa
314034,2021,CONTADURIA PUBLICA,BOGOTÁ,Entre 4 millones y menos de 5.5 millones,Más de 30 horas,Estrato 4,Si,Secundaria (Bachillerato) completa,Si,No,N,Si,Si,Primaria completa
418461,2018,LICENCIATURA EN IDIOMAS EXTRANJEROS,ATLANTICO,Entre 500 mil y menos de 1 millón,Entre 21 y 30 horas,Estrato 4,Si,Educación profesional completa,Si,Si,N,Si,Si,Educación profesional completa
521356,2019,LICENCIATURA EN MATEMATICAS,SUCRE,Menos de 500 mil,0,Estrato 1,No,Ninguno,No,No,N,No,No,Ninguno
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
457734,2018,ECONOMIA,BOGOTÁ,Más de 7 millones,0,Estrato 5,Si,Postgrado,Si,Si,N,No,Si,Educación profesional incompleta
329976,2018,MERCADOLOGIA,BOGOTÁ,Entre 1 millón y menos de 2.5 millones,Más de 30 horas,Estrato 2,Si,Secundaria (Bachillerato) completa,Si,Si,N,Si,Si,Primaria completa
55010,2018,ADMINISTRACION AMBIENTAL Y DE LOS RECURSOS NAT...,BOGOTÁ,Entre 2.5 millones y menos de 4 millones,Entre 21 y 30 horas,Estrato 1,Si,Secundaria (Bachillerato) completa,Si,No,N,No,Si,Ninguno
194187,2019,PSICOLOGIA,BOYACA,Entre 5.5 millones y menos de 7 millones,Entre 21 y 30 horas,Estrato 2,No,Primaria completa,No,No,N,No,Si,Primaria completa


In [168]:
from sklearn.metrics import accuracy_score

preds = pipe.predict(X_test)
print(accuracy_score(preds, y_test))

0.3452057761732852
